In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
import undetected_chromedriver as uc

# Manthan's Data

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}
base_url = ['https://www.espncricinfo.com/series/ipl-2016-968923/match-schedule-fixtures-and-results',
            'https://www.espncricinfo.com/series/ipl-2017-1078425/match-schedule-fixtures-and-results',
            'https://www.espncricinfo.com/series/ipl-2018-1131611/match-schedule-fixtures-and-results',
            'https://www.espncricinfo.com/series/ipl-2019-1165643/match-schedule-fixtures-and-results',
            'https://www.espncricinfo.com/series/ipl-2020-21-1210595/match-schedule-fixtures-and-results']
soups = []
for s in base_url:
    response = requests.get(s,headers=headers)
    soups.append(BeautifulSoup(response.text,'html.parser'))
    print(response.status_code)

200
200
200
200
200


In [4]:
allseasonmatches = []

for i,soup in enumerate(soups) :
    seasonmatch = []
    anchors = soup.find_all('a',class_='ds-inline-flex ds-items-start ds-leading-none')
    for anchor in anchors :
        link = anchor.get('href')
        link = urljoin(base_url[i],link)
        if 'live-cricket-score' in link :
            seasonmatch.append(link)
    allseasonmatches.append(seasonmatch)
allseasonmatches

[['https://www.espncricinfo.com/series/ipl-2016-968923/mumbai-indians-vs-rising-pune-supergiants-1st-match-980901/live-cricket-score',
  'https://www.espncricinfo.com/series/ipl-2016-968923/kolkata-knight-riders-vs-delhi-daredevils-2nd-match-980903/live-cricket-score',
  'https://www.espncricinfo.com/series/ipl-2016-968923/kings-xi-punjab-vs-gujarat-lions-3rd-match-980905/live-cricket-score',
  'https://www.espncricinfo.com/series/ipl-2016-968923/royal-challengers-bangalore-vs-sunrisers-hyderabad-4th-match-980907/live-cricket-score',
  'https://www.espncricinfo.com/series/ipl-2016-968923/kolkata-knight-riders-vs-mumbai-indians-5th-match-980909/live-cricket-score',
  'https://www.espncricinfo.com/series/ipl-2016-968923/gujarat-lions-vs-rising-pune-supergiants-6th-match-980911/live-cricket-score',
  'https://www.espncricinfo.com/series/ipl-2016-968923/delhi-daredevils-vs-kings-xi-punjab-7th-match-980913/live-cricket-score',
  'https://www.espncricinfo.com/series/ipl-2016-968923/sunrisers

Columns to fill
1. Match type
2. POTM
3. Venue
4. Team1
5. Team2
6. Toss Winner
7. Toss Decision
8. Winner
9. Win margin
10. Result type(number of wickets left or run difference)
11. Target runs

## Match Details

In [5]:
def scraper(i,matchlink) :
    try :
        match_data = {
                        'Match Type': '',
                        'POTM': '',
                        'Venue': '',
                        'Team1': '',
                        'Team2': '',
                        'Toss_Winner': '',
                        'Toss_Decision': '',
                        'Winner': '',
                        'Win_Margin': '',
                        'Result_Type': '',
                        'Target_Runs': '',
                        'Season':'',
                        'Match':''
                }
    
        match_data['Season'] = f"{2016+i}"
                
        match = requests.get(matchlink,headers=headers)
        #print(match.status_code)
        soup_match = BeautifulSoup(match.text,'html.parser')
    
        driver.get(matchlink)
    
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
    
        elements = soup.find_all('div', class_='ds-text-tight-m ds-font-regular ds-text-typo-mid3')[0].get_text(strip=True)
        potmscrape = soup.find_all('span', class_='ds-text-tight-m ds-font-medium ds-text-typo ds-underline ds-decoration-ui-stroke hover:ds-text-typo-primary hover:ds-decoration-ui-stroke-primary ds-block ds-cursor-pointer')[0].get_text(strip=True)
        teamscrape = soup.find_all('span',class_='ds-text-tight-l ds-font-bold ds-text-typo hover:ds-text-typo-primary ds-block ds-truncate')
        tossscrape = soup.find_all('td',class_='ds-text-typo')[0].find('span',class_='ds-text-tight-s ds-font-regular').get_text(strip=True)
    
        #  Match type, POTM and Venue
        
        venue = elements.split(',')[1].strip()
        match_data['Venue'] = venue
    
        matching_word = next((word for word in ['Final','Qualifier 1','Eliminator','Qualifier 2'] if word in elements), None)
        
        if matching_word:
            match_data['Match Type'] = matching_word
            match_data['Match'] = matching_word
        else:
            match_data['Match Type'] = 'League'
            match_data['Match'] = elements.split()[:2]
        match_data['POTM'] = potmscrape
        
        # Teams
        
        match_data['Team1'] = teamscrape[0].get_text(strip=True)
        match_data['Team2'] = teamscrape[1].get_text(strip=True)
        
        # Toss Winner and Decision
        
        match_data['Toss_Winner'] = tossscrape.split(',')[0]
        if 'bat' in tossscrape :    
            match_data['Toss_Decision'] = 'bat'
        else :
            match_data['Toss_Decision'] = 'field'
        
        ## Win Details
        
        winscrape = soup.find('p',class_='ds-text-tight-s ds-font-medium ds-truncate ds-text-typo').find('span').get_text(strip=True)
        match_data['Winner'] = winscrape.split()[0]
        if 'wicket' in winscrape : 
            match_data['Result_Type'] = 'wicket'
        else :
            match_data['Result_Type'] = 'runs'
                
        match_data['Win_Margin'] = re.findall(r'\d+', winscrape)[0]
                
        ## Target Runs
                
        trunscrape = soup.find_all('div',class_='ds-text-compact-m ds-text-typo ds-text-right ds-whitespace-nowrap')[0].find('strong').get_text(strip=True)
        match_data['Target_Runs'] = trunscrape.split('/')[0]
    except :
        print(matchlink)
    return match_data

In [6]:

# asmd - all season match data
# smd - season match data
asmd = []
failed = [[],[],[],[],[]]

# Launch stealthy driver
driver = webdriver.Chrome()

for i,seasonmatch in enumerate(allseasonmatches):
    smd = []
    for j,matchlink in enumerate(seasonmatch):
        try :
            match_data = {
                    'Match Type': '',
                    'POTM': '',
                    'Venue': '',
                    'Team1': '',
                    'Team2': '',
                    'Toss_Winner': '',
                    'Toss_Decision': '',
                    'Winner': '',
                    'Win_Margin': '',
                    'Result_Type': '',
                    'Target_Runs': '',
                    'Season':'',
                    'Match':''
            }

            match_data['Season'] = f"{2016+i}"
            
            match = requests.get(matchlink,headers=headers)
            #print(match.status_code)
            soup_match = BeautifulSoup(match.text,'html.parser')

            driver.get(matchlink)
    
            html = driver.page_source
            soup = BeautifulSoup(html, 'lxml')
    
            elements = soup.find_all('div', class_='ds-text-tight-m ds-font-regular ds-text-typo-mid3')[0].get_text(strip=True)
            potmscrape = soup.find_all('span', class_='ds-text-tight-m ds-font-medium ds-text-typo ds-underline ds-decoration-ui-stroke hover:ds-text-typo-primary hover:ds-decoration-ui-stroke-primary ds-block ds-cursor-pointer')[0].get_text(strip=True)
            teamscrape = soup.find_all('span',class_='ds-text-tight-l ds-font-bold ds-text-typo hover:ds-text-typo-primary ds-block ds-truncate')
            tossscrape = soup.find_all('td',class_='ds-text-typo')[0].find('span',class_='ds-text-tight-s ds-font-regular').get_text(strip=True)
    
            #  Match type, POTM and Venue
            
            venue = elements.split(',')[1].strip()
            match_data['Venue'] = venue
    
            matching_word = next((word for word in ['Final','Qualifier 1','Eliminator','Qualifier 2'] if word in elements), None)
            
            if matching_word:
                match_data['Match Type'] = matching_word
                match_data['Match'] = matching_word
            else:
                match_data['Match Type'] = 'League'
                match_data['Match'] = elements.split()[:2]
            match_data['POTM'] = potmscrape
            
            # Teams
            
            match_data['Team1'] = teamscrape[0].get_text(strip=True)
            match_data['Team2'] = teamscrape[1].get_text(strip=True)
            
            # Toss Winner and Decision
            
            match_data['Toss_Winner'] = tossscrape.split(',')[0]
            if 'bat' in tossscrape :    
                match_data['Toss_Decision'] = 'bat'
            else :
                match_data['Toss_Decision'] = 'field'
            
            ## Win Details
            
            winscrape = soup.find('p',class_='ds-text-tight-s ds-font-medium ds-truncate ds-text-typo').find('span').get_text(strip=True)
            match_data['Winner'] = winscrape.split()[0]
            if 'wicket' in winscrape : 
                match_data['Result_Type'] = 'wicket'
            else :
                match_data['Result_Type'] = 'runs'
            
            match_data['Win_Margin'] = re.findall(r'\d+', winscrape)[0]
            
            ## Target Runs
            
            trunscrape = soup.find_all('div',class_='ds-text-compact-m ds-text-typo ds-text-right ds-whitespace-nowrap')[0].find('strong').get_text(strip=True)
            match_data['Target_Runs'] = trunscrape.split('/')[0]
        except :
            failed[i].append(matchlink)
        smd.append(match_data)
    asmd.append(smd)
driver.quit()
print(failed)

[[], ['https://www.espncricinfo.com/series/ipl-2017-1078425/royal-challengers-bangalore-vs-sunrisers-hyderabad-29th-match-1082619/live-cricket-score', 'https://www.espncricinfo.com/series/ipl-2017-1078425/gujarat-lions-vs-mumbai-indians-35th-match-1082625/live-cricket-score'], [], ['https://www.espncricinfo.com/series/ipl-2019-1165643/delhi-capitals-vs-kolkata-knight-riders-10th-match-1175365/live-cricket-score', 'https://www.espncricinfo.com/series/ipl-2019-1165643/royal-challengers-bangalore-vs-rajasthan-royals-49th-match-1178424/live-cricket-score', 'https://www.espncricinfo.com/series/ipl-2019-1165643/mumbai-indians-vs-sunrisers-hyderabad-51st-match-1178426/live-cricket-score'], ['https://www.espncricinfo.com/series/ipl-2020-21-1210595/delhi-capitals-vs-kings-xi-punjab-2nd-match-1216493/live-cricket-score', 'https://www.espncricinfo.com/series/ipl-2020-21-1210595/royal-challengers-bangalore-vs-mumbai-indians-10th-match-1216547/live-cricket-score', 'https://www.espncricinfo.com/seri

In [7]:
frames = []
for smd in asmd :
    df = pd.DataFrame(smd)
    frames.append(df)
result = pd.concat(frames).reset_index()
result.to_csv('data.csv')
result

,index,Match Type,POTM,Venue,Team1,Team2,Toss_Winner,Toss_Decision,Winner,Win_Margin,Result_Type,Target_Runs,Season,Match
0,0,League,Ajinkya Rahane,Wankhede,MI,RPS,Mumbai Indians,bat,Supergiants,9,wicket,121,2016,"[1st, match]"
1,1,League,Andre Russell,Eden Gardens,DC,KKR,Kolkata Knight Riders,field,KKR,9,wicket,98,2016,"[2nd, match]"
2,2,League,Aaron Finch,Mohali,KXIP,GL,Gujarat Lions,field,Guj,5,wicket,161,2016,"[3rd, match]"
3,3,League,AB de Villiers,Bengaluru,RCB,SRH,Sunrisers Hyderabad,field,RCB,45,runs,227,2016,"[4th, match]"
4,4,League,Rohit Sharma,Eden Gardens,KKR,MI,Mumbai Indians,field,MI,6,wicket,187,2016,"[5th, match]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,55,League,Shahbaz Nadeem,Sharjah,MI,SRH,Sunrisers Hyderabad,field,SRH,10,wicket,149,2020,"[56th, Match]"
296,56,Qualifier 1,Jasprit Bumrah,Dubai (DICS),MI,DC,Delhi Capitals,field,MI,57,runs,200,2020,Qualifier 1
297,57,Eliminator,Kane Williamson,Abu Dhabi,RCB,SRH,Sunrisers Hyderabad,field,SRH,6,wicket,131,2020,Eliminator
298,58,Qualifier 2,Marcus Stoinis,Abu Dhabi,DC,SRH,Delhi Capitals,bat,DC,17,runs,189,2020,Qualifier 2


In [8]:
driver = webdriver.Chrome()
miss = []
for i,season in enumerate(failed) :
    for matchlink in season :
        if matchlink :
            miss.append(scraper(i,matchlink))
driver.quit()
missdf = pd.DataFrame(miss)
missdf

https://www.espncricinfo.com/series/ipl-2017-1078425/royal-challengers-bangalore-vs-sunrisers-hyderabad-29th-match-1082619/live-cricket-score
https://www.espncricinfo.com/series/ipl-2017-1078425/gujarat-lions-vs-mumbai-indians-35th-match-1082625/live-cricket-score
https://www.espncricinfo.com/series/ipl-2019-1165643/delhi-capitals-vs-kolkata-knight-riders-10th-match-1175365/live-cricket-score
https://www.espncricinfo.com/series/ipl-2019-1165643/royal-challengers-bangalore-vs-rajasthan-royals-49th-match-1178424/live-cricket-score
https://www.espncricinfo.com/series/ipl-2019-1165643/mumbai-indians-vs-sunrisers-hyderabad-51st-match-1178426/live-cricket-score
https://www.espncricinfo.com/series/ipl-2020-21-1210595/delhi-capitals-vs-kings-xi-punjab-2nd-match-1216493/live-cricket-score
https://www.espncricinfo.com/series/ipl-2020-21-1210595/royal-challengers-bangalore-vs-mumbai-indians-10th-match-1216547/live-cricket-score
https://www.espncricinfo.com/series/ipl-2020-21-1210595/sunrisers-hyd

,Match Type,POTM,Venue,Team1,Team2,Toss_Winner,Toss_Decision,Winner,Win_Margin,Result_Type,Target_Runs,Season,Match
0,,,,,,,,,,,,2017,
1,League,Krunal Pandya,Rajkot,GL,MI,Gujarat Lions,bat,Match,,runs,,2017,"[35th, match]"
2,League,Prithvi Shaw,Delhi,KKR,DC,Delhi Capitals,field,Match,,runs,,2019,"[10th, Match]"
3,,,,,,,,,,,,2019,
4,League,Jasprit Bumrah,Wankhede,MI,SRH,Mumbai Indians,bat,Match,,runs,,2019,"[51st, Match]"
5,League,Marcus Stoinis,Dubai (DICS),DC,KXIP,Kings XI Punjab,field,Match,,runs,,2020,"[2nd, Match]"
6,League,AB de Villiers,Dubai (DICS),RCB,MI,Mumbai Indians,field,Match,,runs,,2020,"[10th, Match]"
7,League,Lockie Ferguson,Abu Dhabi,KKR,SRH,Sunrisers Hyderabad,field,Match,,runs,,2020,"[35th, Match,]"


In [11]:
empty_rows = result[result['Match Type'].isna()].index
for idx, miss_row in zip(empty_rows, missdf.iterrows()):
    result.loc[idx] = miss_row[1]
result.to_csv('data_2016-2020.csv')

In [10]:
result[result['Match Type']=='']['Match Type'].count()

np.int64(2)